In [205]:
with open('data/tiny-shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [206]:
print('length of dataset in characters: ', len(text))

length of dataset in characters:  1115394


In [207]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [208]:
# create a batch of 1000 characters and build a vocabulary
char = sorted(list(set(text)))
vocab_size = len(char)
print(''.join(char))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [209]:
# encode the characters to intergers and vice versa
stoi = {c:i for i,c in enumerate(char)}
itos = {i:c for i,c in enumerate(char)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda i: ''.join([itos[c] for c in i])

In [210]:
print(encode('Vishal Chandru'))
print(decode(encode('Vishal Chandru')))

[34, 47, 57, 46, 39, 50, 1, 15, 46, 39, 52, 42, 56, 59]
Vishal Chandru


In [211]:
# Use tiktoken
import tiktoken
enc = tiktoken.get_encoding('gpt2')
enc.n_vocab

50257

In [212]:
enc.encode("Vishal Chandru")

[53, 680, 282, 20631, 622]

In [213]:
enc.decode([53, 680, 282, 20631, 622])

'Vishal Chandru'

In [214]:
# Encode the entire text dataset and use pytorch to store the encoded text in tensor
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [215]:
# Lets split the data into train and validation
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [216]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [217]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
print(x,'\n')
print(y, '\n')

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"{context} ----> {target}")

tensor([18, 47, 56, 57, 58,  1, 15, 47]) 

tensor([47, 56, 57, 58,  1, 15, 47, 58]) 

tensor([18]) ----> 47
tensor([18, 47]) ----> 56
tensor([18, 47, 56]) ----> 57
tensor([18, 47, 56, 57]) ----> 58
tensor([18, 47, 56, 57, 58]) ----> 1
tensor([18, 47, 56, 57, 58,  1]) ----> 15
tensor([18, 47, 56, 57, 58,  1, 15]) ----> 47
tensor([18, 47, 56, 57, 58,  1, 15, 47]) ----> 58


In [218]:
torch.manual_seed(1234)
batch_size = 4
block_size = 8

def get_batch(split_type):
    data = train_data if split_type == 'train' else val_data
    idx = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in idx])
    y = torch.stack([data[i+1:i+block_size+1] for i in idx])
    return x,y

x_batch, y_batch = get_batch("train")
print("Inputs :")
print(x_batch.shape)
print(x_batch)
print('\n')
print("Targets :")
print(y_batch.shape)
print(y_batch)
print('\n')

print('-'*30)

for b in range(batch_size):
    for t in range(block_size):
        context = x_batch[b, :t+1]
        target = y_batch[b,t]
        print(f"{context.tolist()} -----> {target}")

Inputs :
torch.Size([4, 8])
tensor([[51, 59, 57, 58,  1, 39, 61, 39],
        [63,  6,  1, 61, 46, 53,  5, 57],
        [39,  1, 41, 46, 47, 50, 42, 10],
        [53, 59, 10,  0, 37, 53, 59,  1]])


Targets :
torch.Size([4, 8])
tensor([[59, 57, 58,  1, 39, 61, 39, 63],
        [ 6,  1, 61, 46, 53,  5, 57,  1],
        [ 1, 41, 46, 47, 50, 42, 10,  0],
        [59, 10,  0, 37, 53, 59,  1, 57]])


------------------------------
[51] -----> 59
[51, 59] -----> 57
[51, 59, 57] -----> 58
[51, 59, 57, 58] -----> 1
[51, 59, 57, 58, 1] -----> 39
[51, 59, 57, 58, 1, 39] -----> 61
[51, 59, 57, 58, 1, 39, 61] -----> 39
[51, 59, 57, 58, 1, 39, 61, 39] -----> 63
[63] -----> 6
[63, 6] -----> 1
[63, 6, 1] -----> 61
[63, 6, 1, 61] -----> 46
[63, 6, 1, 61, 46] -----> 53
[63, 6, 1, 61, 46, 53] -----> 5
[63, 6, 1, 61, 46, 53, 5] -----> 57
[63, 6, 1, 61, 46, 53, 5, 57] -----> 1
[39] -----> 1
[39, 1] -----> 41
[39, 1, 41] -----> 46
[39, 1, 41, 46] -----> 47
[39, 1, 41, 46, 47] -----> 50
[39, 1, 41, 46, 47, 

In [219]:
a = torch.randint(200, (4,))
print(a)
torch.stack([data[i:i+8] for i in a])

tensor([116, 164,  32,  25])


tensor([[42,  1, 56, 39, 58, 46, 43, 56],
        [56, 43, 57, 53, 50, 60, 43, 42],
        [ 1, 39, 52, 63,  1, 44, 59, 56],
        [54, 56, 53, 41, 43, 43, 42,  1]])

In [220]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1234)

class Transformer(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token reads off teh logits for the next token from the embedding table
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
    
    def forward(self, idx, targets=None):

        # idx and targets are both (B, T) tensor of integer
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape 
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits,loss = self(idx) # (B, T, C)
            logits = logits[:, -1, :] # (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim = 1)

        return idx

model = Transformer(vocab_size)
logits, loss = model(x_batch, y_batch)
print(logits.shape)
print(loss)

print(decode(model.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens = 100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8650, grad_fn=<NllLossBackward0>)

UkBP-NBU& Y!lL ey-CofWUelZ?Jz.saHcpq!Uo's$VR&mP'xOXt3fTxwKb$!?$Kn?,aUs'ao3Jm:QcJwp
K!Uh:mEkwOssPcDIU


In [221]:
# PyTorch Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3)

In [222]:
batch_size = 32

for steps in range(10000):

    # sample a batch
    x_batch, y_batch = get_batch('train')

    # evaluate the loss
    logits, loss = model(x_batch, y_batch)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5062272548675537


In [223]:
print(decode(model.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens = 500)[0].tolist()))


Whe INRDins'l Of orayen If ene tant ay fr IVOLUThth d I onge mofondiathety o h, bis.


Merouderut, ldr cain. is theat the, lllinth me, hengherd s,
FFFFringhe orse ay HoHAE:
ADI wathoserpowhru MASARYeoloocast me andsvo b'd;ler, t.
O: se pengis:
WADUScrt inu I th:


Anpengs th g, aror n; byo's cherowingue wakeron,
th.
Wind, thave Kmendlsourvese m pr?
My w
I akers wha$THMu,
BuME:


at I'dendslot ut is m;; med o.
NUThanin hepevisispotwigod'l,
Yo y, finoourghare!
's hitfonyoan ireno psoravid b ge nd 


## Mathematical trick for self attention

In [224]:
# self attention
torch.manual_seed(1234)
B, T, C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [225]:
# version 1: use for loop (inefficient)
# x[b, t] = mean_{i<=t} x[b, i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t, C)
        xbow[b,t] = torch.mean(xprev, 0)

In [226]:
x[0]

tensor([[-0.1117, -0.4966],
        [ 0.1631, -0.8817],
        [ 0.0539,  0.6684],
        [-0.0597, -0.4675],
        [-0.2153,  0.8840],
        [-0.7584, -0.3689],
        [-0.3424, -1.4020],
        [ 0.3206, -1.0219]])

In [227]:
xbow[0]

tensor([[-0.1117, -0.4966],
        [ 0.0257, -0.6891],
        [ 0.0351, -0.2366],
        [ 0.0114, -0.2944],
        [-0.0339, -0.0587],
        [-0.1547, -0.1104],
        [-0.1815, -0.2949],
        [-0.1187, -0.3858]])

In [228]:
torch.manual_seed(1234)
a = torch.ones(3,3)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a = ')
print(a)
print('-----')
print('b = ')
print(b)
print('-----')
print('c = ')
print(c)
print('-----')

a = 
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
-----
b = 
tensor([[5., 1.],
        [6., 5.],
        [6., 4.]])
-----
c = 
tensor([[17., 10.],
        [17., 10.],
        [17., 10.]])
-----


In [229]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [230]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a = ')
print(a)
print('-----')
print('b = ')
print(b)
print('-----')
print('c = ')
print(c)
print('-----')

a = 
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
-----
b = 
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
-----
c = 
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
-----


In [231]:
# version 2: use matrix mul
weight = torch.tril(torch.ones(T, T))
weight = weight / weight.sum(1, keepdim=True)
xbow2 = weight @ x # (T, T) @ (B, T, C) ----> (B, T, T) @ (B, T, C) -----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [232]:
xbow[0]

tensor([[-0.1117, -0.4966],
        [ 0.0257, -0.6891],
        [ 0.0351, -0.2366],
        [ 0.0114, -0.2944],
        [-0.0339, -0.0587],
        [-0.1547, -0.1104],
        [-0.1815, -0.2949],
        [-0.1187, -0.3858]])

In [233]:
xbow2[0]

tensor([[-0.1117, -0.4966],
        [ 0.0257, -0.6891],
        [ 0.0351, -0.2366],
        [ 0.0114, -0.2944],
        [-0.0339, -0.0587],
        [-0.1547, -0.1104],
        [-0.1815, -0.2949],
        [-0.1187, -0.3858]])

In [234]:
# version 3: use softmax
tril = torch.tril(torch.ones(T,T))
weight = torch.zeros((T,T))
weight = weight.masked_fill(tril == 0, float('-inf'))
weight = F.softmax(weight, dim = -1)
xbow3 = weight @ x
torch.allclose(xbow, xbow3)

True

In [235]:
# version 4: self-attention
torch.manual_seed(1234)
B, T, C = 4, 8, 32

x = torch.randn(B,T,C)

# single head self-attention
head_size = 16

key = nn.Linear(C, head_size, bias = False)
query = nn.Linear(C, head_size, bias = False)
value = nn.Linear(C, head_size, bias = False)

k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2,-1) # (B, T, 16) @ (B, 16, T) ----> (B, T, T)

tril = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim = -1)

v = value(x)
out = wei @ v
# out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [236]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2767, 0.7233, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.7186, 0.0726, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1615, 0.0213, 0.5879, 0.2292, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0558, 0.0951, 0.1961, 0.1000, 0.5530, 0.0000, 0.0000, 0.0000],
        [0.0935, 0.6474, 0.0659, 0.0296, 0.0822, 0.0814, 0.0000, 0.0000],
        [0.0367, 0.0325, 0.0941, 0.4913, 0.2240, 0.0431, 0.0784, 0.0000],
        [0.2329, 0.2915, 0.0324, 0.1427, 0.0200, 0.0800, 0.0436, 0.1569]],
       grad_fn=<SelectBackward0>)

In [247]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2,-1)

In [248]:
print(k.var())
print(q.var())
print(wei.var())

tensor(0.9683)
tensor(0.9970)
tensor(17.0821)


In [249]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2,-1) * head_size**-0.5

In [250]:
print(k.var())
print(q.var())
print(wei.var())

tensor(1.0466)
tensor(1.0203)
tensor(1.1102)
